# Deploy TensorFlow model to ACI


In the previous lab, we used `Hyperdrive` to tune FCNN top model. The model was registered in Model Registry.

Now, we are ready to deploy the model.

The model can be deployed to a variety of target runtimes, including:
- Azure Container Instance
- Azure Kubernetes Service
- IoT Edge
- FPGA


In this lab, we will deploy the model as a web service in Azure Container Instance.

![AML Arch](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/amlarch.png)

## Connect to the workspace

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.8


In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()

Found the config file in: /home/nbuser/library/aml_config/config.json


## Deploy as web service

To build the correct environment for ACI, provide the following:
* A scoring script that invokes the model
* An environment file to show what packages need to be installed
* A configuration file to build the ACI
* The model you trained before


### Create scoring script

Create the scoring script, called score.py, used by the web service call to invoke the model.

You must include two required functions in the scoring script:
* The `init()` function, which loads the model into a global object. This function is run only once when the Docker container is started. 

* The `run(input_data)` function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats can be used.

In [3]:
%%writefile score.py
import json
import os
import numpy as np
import random
import tensorflow as tf

from tensorflow.keras.applications import resnet50
from tensorflow.keras.preprocessing import image

from azureml.core.model import Model
from azureml.core import Workspace

def init():
    try:
        
        # Create ResNet50 featurizer
        global featurizer
        
        featurizer = resnet50.ResNet50(
            weights = 'imagenet', 
            input_shape=(224,224,3), 
            include_top = False,
            pooling = 'avg')
        
        # Load top model
        global model   

        model_name = '<<modelid>>'
        model_path = Model.get_model_path(model_name)
        model = tf.keras.models.load_model(model_path)
        
    except Exception as e:
        print('Exception during init: ', str(e))

  

def run(raw_data):
    try:
        # convert json to numpy array
        images = np.array(json.loads(raw_data)['data'])
        # normalize as required by ResNet50
        images = resnet50.preprocess_input(images)
        # Extract bottleneck featurs
        features = featurizer.predict(images)
        # Make prediction
        predictions = model.predict(features)
        
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})
    
    # Return both numeric and string predictions
    # return json.dumps({"predictions": predictions.tolist(), "labels": string_predictions})
    return json.dumps({"predictions": predictions.tolist()})

Overwriting score.py


Substitute the actual model ID in the script file.

In [4]:
from azureml.core.model import Model

model_name = 'aerial_classifier'
model = Model(ws, name=model_name)
script_file_name = 'score.py'

with open(script_file_name, 'r') as cefr:
    content = cefr.read()
    
with open(script_file_name, 'w') as cefw:
    cefw.write(content.replace('<<modelid>>', model.name))

Review the updated script.

In [5]:
with open("score.py","r") as f:
    print(f.read())

import json
import os
import numpy as np
import random
import tensorflow as tf

from tensorflow.keras.applications import resnet50
from tensorflow.keras.preprocessing import image

from azureml.core.model import Model
from azureml.core import Workspace

def init():
    try:
        
        # Create ResNet50 featurizer
        global featurizer
        
        featurizer = resnet50.ResNet50(
            weights = 'imagenet', 
            input_shape=(224,224,3), 
            include_top = False,
            pooling = 'avg')
        
        # Load top model
        global model   

        model_name = 'aerial_classifier'
        model_path = Model.get_model_path(model_name)
        model = tf.keras.models.load_model(model_path)
        
    except Exception as e:
        print('Exception during init: ', str(e))

  

def run(raw_data):
    try:
        # convert json to numpy array
        images = np.array(json.loads(raw_data)['data'])
        # normalize as required by ResNet50
    

### Create a Conda dependencies environment file.

Next, create an environment file that specifies the script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image. 




In [6]:
from azureml.core.conda_dependencies import CondaDependencies 

mycondaenv = CondaDependencies()
mycondaenv.add_pip_package("tensorflow")
mycondaenv.add_pip_package("h5py")
mycondaenv.add_pip_package("pillow")
mycondaenv.add_pip_package("scipy")

with open("mydeployenv.yml","w") as f:
    f.write(mycondaenv.serialize_to_string())

Review the content of 'yml' file.

In [7]:
with open("mydeployenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults
  - tensorflow
  - h5py
  - pillow
  - scipy



### Create docker image for deployment

To create a Container Image, you need four things: the model metadata (as retrieved from Model Registry), the scoring script file, the runtime configuration (defining whether Python or PySpark should be used) and the Conda Dependencies file.

In [8]:
from azureml.core.image import ContainerImage, Image

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="mydeployenv.yml",
                                                  description="Image for aerial classifier",
                                                  tags={"Classifier": "FCNN"})

image = Image.create(name = "aerial-classifier-image",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running.................................
SucceededImage creation operation finished for image aerial-classifier-image:1, operation "Succeeded"


### Define ACI configuration

Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container. The default is 1 core and 1 gigabyte of RAM. Since we are using ResNet50 featurizer we are CPU bound.  In this lab we will use the defaults but you should always go through the proper performance plannig exercise to find the right configuration.

In [9]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "aerial",  "method" : "classifier"}, 
                                               description='Predict aerial images')

### Deploy in ACI

Deploy the image as a web service in Azure Container Instance.

In [10]:
from azureml.core.webservice import Webservice

aci_service_name = 'aerial-classifier-svc'
print(aci_service_name)
aci_service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                           image = image,
                                           name = aci_service_name,
                                           workspace = ws)
aci_service.wait_for_deployment(True)

aerial-classifier-svc
Creating service
Running...................
SucceededACI service creation operation finished, operation "Succeeded"


To troubleshoot any deployment issues you can retrieve deployment logs.

In [11]:
print(aci_service.get_logs())

2019-02-04T17:47:20,042734137+00:00 - iot-server/run 
ok: run: nginx: (pid 10) 0s
ok: run: rsyslog: (pid 11) 0s
2019-02-04T17:47:20,049472888+00:00 - gunicorn/run 
ok: run: rsyslog: (pid 11) 0s
2019-02-04T17:47:20,050763098+00:00 - rsyslog/run 
2019-02-04T17:47:20,050680497+00:00 - nginx/run 
2019-02-04T17:47:20,058477756+00:00 - iot-server/finish 3 0
2019-02-04T17:47:20,062779189+00:00 - Exit code 3 is not normal. Restarting iot-server.
ok: run: rsyslog: (pid 11) 0s
{"timestamp": "2019-02-04T17:47:20.338201Z", "message": "Starting gunicorn 19.6.0", "host": "wk-caas-c77f5ec4becd415a88f32b3560849833-092f74c6a537272167d75e", "path": "/opt/miniconda/lib/python3.6/site-packages/gunicorn/glogging.py", "tags": "%(module)s, %(asctime)s, %(levelname)s, %(message)s", "level": "INFO", "logger": "gunicorn.error", "msg": "Starting gunicorn %s", "stack_info": null}
{"timestamp": "2019-02-04T17:47:20.339114Z", "message": "Listening at: http://127.0.0.1:9090 (15)", "host": "wk-caas-c77f5ec4becd415a88

Get the scoring web service's HTTP endpoint, which accepts REST client calls. This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.

In [12]:
print(aci_service.scoring_uri)

http://52.157.21.30:80/score


## Test deployed service


Download test images.

In [13]:
%%sh
cd /tmp
wget -nv https://azureailabs.blob.core.windows.net/aerialsamples/barren-1.png
wget -nv https://azureailabs.blob.core.windows.net/aerialsamples/cultivated-1.png
wget -nv https://azureailabs.blob.core.windows.net/aerialsamples/developed-1.png
ls -l


total 308
-rw-r--r-- 1 nbuser nbuser 71190 Nov  6 01:19 barren-1.png
-rw------- 1 nbuser nbuser    54 Jan 24 07:30 clock_gettimeedtUjZ.c
-rw------- 1 nbuser nbuser    54 Jan 24 07:50 clock_gettimemz9wcit0.c
-rw------- 1 nbuser nbuser    54 Jan 24 07:39 clock_gettimeno97sd6l.c
-rw------- 1 nbuser nbuser    54 Jan 24 07:30 clock_gettimevxIYQE.c
-rw------- 1 nbuser nbuser    54 Jan 24 07:39 clock_gettimew7jo8kez.c
-rw------- 1 nbuser nbuser    54 Jan 24 07:50 clock_gettimezkrgfhja.c
-rw-r--r-- 1 nbuser nbuser 79841 Nov  6 01:19 cultivated-1.png
-rw-r--r-- 1 nbuser nbuser 96252 Nov  6 01:19 developed-1.png
drwxr-xr-x 2 root   root    4096 Jan 24 07:22 hsperfdata_root
drwxr-xr-x 3 root   root    4096 Jan 24 06:27 npm-169-867310c7
drwxr-xr-x 3 root   root    4096 Jan 24 06:28 npm-293-63cc6e69
drwxr-xr-x 3 root   root    4096 Jan 24 06:25 npm-45-1d593cbf
drwxr-xr-x 2 nbuser nbuser  4096 Jan 24 07:58 paket
-rw------- 1 nbuser nbuser    43 Feb  4 17:42 requirements0g46kwai.txt
-rw-r--r-- 1 nbus

2019-02-04 17:51:15 URL:https://azureailabs.blob.core.windows.net/aerialsamples/barren-1.png [71190/71190] -> "barren-1.png" [1]
2019-02-04 17:51:16 URL:https://azureailabs.blob.core.windows.net/aerialsamples/cultivated-1.png [79841/79841] -> "cultivated-1.png" [1]
2019-02-04 17:51:17 URL:https://azureailabs.blob.core.windows.net/aerialsamples/developed-1.png [96252/96252] -> "developed-1.png" [1]


Define utility function that wraps loading images and invoking the service. The function takes as an input a list of pathnames to images. It loads and converts the binary images into a JSON array and invokes the service passing the JSON payload.

In [14]:
from PIL import Image
import numpy as np
import json

def score(pathnames, service):
    images = []
    for pathname in pathnames:
        img = Image.open(pathname)
        img = np.asarray(img).tolist()
        images.append(img)
    images = json.dumps({"data": images})
    images = bytes(images, encoding='utf8')
    results = json.loads(service.run(input_data=images))
    return results

Call the service.

In [15]:
aci_service = Webservice(workspace=ws, name='aerial-classifier-svc')

pathnames = ['/tmp/barren-1.png', '/tmp/developed-1.png', '/tmp/cultivated-1.png']

results = score(pathnames, aci_service)

print(results)

{'predictions': [[0.010549367405474186, 0.015427099540829659, 0.0005459703388623893, 0.0008418725919909775, 0.024088528007268906, 0.9485471844673157], [1.100195348158195e-07, 1.7579907307663234e-06, 0.9999979734420776, 8.429084985550617e-09, 1.2771913304732152e-07, 2.627759954076936e-10], [6.1513578657468315e-06, 0.9999005794525146, 6.986770313233137e-05, 1.1953638932027388e-05, 1.1384202480257954e-05, 2.2283416001300793e-08]]}


## Clean up resources

Delete the ACI service.

In [17]:
aci_service.delete()

No service with name aerial-classifier-svc found to delete.
